# iPython Nutrition Notebook

This notebook was created to make it easier to understand how several behavioral variables affect your body's composition. Please add or remove from it what you like, as it was only ever intended to aid the otherwise obnoxiously tedious activity that is macro tracking.

## Initial setup
Start MongoDB and run this cell before interacting with the rest of this notebook.

In [22]:
from pymongo import MongoClient
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt

#Start MongoDB Client
client = MongoClient()
db = client.primer
coll = db.dataset

#FOODS

#Simple Foods
empty = np.array([0.,0.,0.,0.])
Egg = np.array([78., 5., 0.6, 6.])
Avocado_whole = np.array([234., 21., 12., 2.9])
Sour_Cream_1tbsp = np.array([23., 2.4, 0.3, 0.2])
Spinach = np.array([7., 0.1, 1.1, 0.9])
Chicken_1oz = np.array([32.5, 0.75, 0, 6.5])
Peanut_Butter_2tbsp = np.array([200., 16., 6., 8.])
Refried_Beans_1serving = np.array([195, 0.89, 33.7, 10.64])
Trail_Mix_hfcup = np.array([360, 24., 31.2, 12.])
Protein_Powder_1scoop = np.array([140., 2., 6., 24.])
Oatmeal_hfcup = np.array([150.0, 3.0, 27.0, 5.0])
Sweet_Potato_1oz = np.array([86./3.5274, 0.1/3.5274, 5.6698985088, 1.6/3.5274])
White_Rice_1cp = np.array([206, 0.4, 45.0, 4.3])
One_Percent_Milk_1cup = np.array([130, 2.5, 16., 11.])
Two_Percent_Milk_1cup = np.array([130., 5., 13., 10.])
Cottage_Cheese_hfcup = np.array([80.0, 0.0, 6.0, 13.0])

#Protein
Amplified_Mass_1scoop = np.array([187.5, 1.5, 31, 12.5])
Protein_Powder_MochaCappuccino = np.array([120, 1.5, 3., 24.])
Protein_Powder_DoubleChocolate = np.array([120, 1., 3., 24.])

#Mealy Foods
chicken_serving_small = 8*Chicken_1oz
chicken_serving_large = 10*Chicken_1oz
halfAnAvocado = np.multiply(0.5, Avocado_whole)
quarterAnAvocado = np.multiply(0.25, Avocado_whole)
omelette = 6*Egg + halfAnAvocado + 6*Sour_Cream_1tbsp + Spinach
omeletteLessFat = 6*Egg + quarterAnAvocado + Spinach
oatmeal = 2*Oatmeal_hfcup
halfMass = 2*Amplified_Mass_1scoop
halfMassWithMilk = halfMass + One_Percent_Milk_1cup

###### Enums ######
#Macronutrients
Fat, Carbohydrates, Protein = range(3)
#Gender
Male, Female = range(2)
#Activity Level
Sedentary, Light, Moderate, VeryActive = range(4)
#Goals
WeightLoss, Maintenance, Bulking = [0.8, 1.0, 1.1]

    
#utility functions
def lbsToKG(weightInLbs):
    return weightInLbs*0.453592

def inToCm(heightInIn):
    return heightInIn*2.54

def calculateDailyCalories(weightInLbs, heightInIn, age, gender, activityLevel, goal):
    restingEnergyExpenditure = 0.
    if gender == Male:
        restingEnergyExpenditure = 10*lbsToKG(weightInLbs) + 6.25*inToCm(heightInIn) - 5*age + 5.
    elif gender == Female:
        restingEnergyExpenditure = 10.*lbsToKG(weightInLbs) + 6.25*inToCm(heightInIn) - 5.*age - 161.
        
    totalDailyEnergyExpenditure = ([1.2, 1.375, 1.55, 1.725][activityLevel]) * restingEnergyExpenditure
    
    if total[0] > totalDailyEnergyExpenditure*goal:
        print "Warning. Over daily calorie budget by " + `total[0] - totalDailyEnergyExpenditure*goal` + " calories."
    
    return totalDailyEnergyExpenditure*goal

def calculateGramsOfMacro(targetCalories, currentMacros, macroString, weightInLbs, activityLevel):
    gramsProtein = 0.0
    gramsCarbs = 0.0
    gramsFat = 0.0
    
    if activityLevel <= 1:
        gramsProtein = weightInLbs*0.825
    else:
        gramsProtein = weightInLbs
    gramsFat = (targetCalories * 0.25)/9.0
    gramsCarbs = (targetCalories - (gramsFat*9.0 + gramsProtein*4.0))/4.0 
    
    returnArray = [gramsFat, gramsCarbs, gramsProtein]
    
    return returnArray[eval(macroString)]

def calculateGramsOfMacroLeft(targetCalories, currentMacros, macroString, weightInLbs, activityLevel):
    gramsProtein = calculateGramsOfMacro(targetCalories, currentMacros, "Protein", weightInLbs, activityLevel)
    gramsCarbs = calculateGramsOfMacro(targetCalories, currentMacros, "Carbohydrates", weightInLbs, activityLevel)
    gramsFat = calculateGramsOfMacro(targetCalories, currentMacros, "Fat", weightInLbs, activityLevel)
    
    gramsProtein = gramsProtein - currentMacros[3]
    gramsCarbs = gramsCarbs - currentMacros[2]  
    gramsFat = gramsFat - currentMacros[1]
    
    caloriesToSubtract = currentMacros[0] - targetCalories
    constantMultiplier = [9,4,4]
    returnArray = [gramsFat, gramsCarbs, gramsProtein]
    
    if caloriesToSubtract > 0:
        return 0
    else:
        return returnArray[eval(macroString)]

def calculateServingsOfFoodLeftByMacro(targetCalories, targetMacroAmountInGrams, currentMacros, macroString, food):
    gramsLeft = 0.0
    servings = 0.0
    
    gramsLeft = targetMacroAmountInGrams - currentMacros[eval(macroString)+1]
    servings = gramsLeft/eval(food)[eval(macroString)+1]
    
    if servings < 0:
        print "Over " + `macroString` + " budget by " + `-gramsLeft` + " grams."
        print "Equivalent to " + `-gramsLeft/eval(food)[eval(macroString)+1]` + " servings of " + `food`
    elif (targetCalories - currentMacros[0]) < 0:
        return 0
    
    return servings
    

## Tracking
This cell can be used for day-to-day tracking.

In [24]:
#Beginning-of-day tracking
weight = 161.4
bodyfatpercentage = 12
sleep = {"hours": 8, "minutes": 24}

#Meal tracking
firstMeal = Protein_Powder_DoubleChocolate + omeletteLessFat
print("First Meal Calories: " + `firstMeal[0]`)
print("First Meal Fat: " + `firstMeal[1]`)
print("First Meal Carbs: " + `firstMeal[2]`)
print("First Meal Protein: " + `firstMeal[3]` + "\n")

secondMeal = 3*oatmeal + 7.9*Chicken_1oz
print("Second Meal Calories: " + `secondMeal[0]`)
print("Second Meal Fat: " + `secondMeal[1]`)
print("Second Meal Carbs: " + `secondMeal[2]`)
print("Second Meal Protein: " + `secondMeal[3]` + "\n")

thirdMeal = 12*Sweet_Potato_1oz
print("Third Meal Calories: " + `thirdMeal[0]`)
print("Third Meal Fat: " + `thirdMeal[1]`)
print("Third Meal Carbs: " + `thirdMeal[2]`)
print("Third Meal Protein: " + `thirdMeal[3]` + "\n")

fourthMeal = empty
print("Fourth Meal Calories: " + `fourthMeal[0]`)
print("Fourth Meal Fat: " + `fourthMeal[1]`)
print("Fourth Meal Carbs: " + `fourthMeal[2]`)
print("Fourth Meal Protein: " + `fourthMeal[3]` + "\n")

total = firstMeal + secondMeal + thirdMeal + fourthMeal
print("Total Calories: " + `total[0]`)
print("Total Fat: " + `total[1]`)
print("Total Carbs: " + `total[2]`)
print("Total Protein: " + `total[3]`)

First Meal Calories: 653.5
First Meal Fat: 36.350000000000001
First Meal Carbs: 10.699999999999999
First Meal Protein: 61.625

Second Meal Calories: 1156.75
Second Meal Fat: 23.925000000000001
Second Meal Carbs: 162.0
Second Meal Protein: 81.349999999999994

Third Meal Calories: 292.5667630549413
Third Meal Fat: 0.34019391052900155
Third Meal Carbs: 68.038782105600006
Third Meal Protein: 5.4431025684640248

Fourth Meal Calories: 0.0
Fourth Meal Fat: 0.0
Fourth Meal Carbs: 0.0
Fourth Meal Protein: 0.0

Total Calories: 2102.8167630549415
Total Fat: 60.615193910529008
Total Carbs: 240.7387821056
Total Protein: 148.41810256846401


## Database Insertion
Use this cell to insert today's data into your MongoDB database

In [5]:
try:
  db, total
except NameError:
  print "Make sure to run the initial setup and tracking cells before this one."
else:
    daysSinceEpoch = (pd.to_datetime('today') - pd.datetime(1970,1,1)).days - 1

    result = db.macros.insert_one(
        {
            "date": daysSinceEpoch,
            "macros": {
                "calories": total[0],
                "fat": total[1],
                "carbohydrates": total[2],
                "protein": total[3]
            },
            "weight": weight,
            "bodyfat%": bodyfatpercentage,
            "sleep": sleep
        }
    )
    print "Insertion successful."

Insertion successful.


In [3]:
db.macros.delete_one({'date': 16861})

## Data retrieval
Use this cell to view all data collected over a 'daysShown'-day period. The graping cell also depends on the output of this cell, so run this one first before graphing your data.

In [6]:
try:
  pd, db
except NameError:
  print "Make sure to run the initial setup cell before this one."
else:
    #lists to store tracked variables
    daysList = []
    caloriesList = []
    fatList = []
    carbsList = []
    proteinList = []
    sleepList = []
    weightList = []

    #setup for mongo search
    daysShown = 9
    earliestDate = (pd.to_datetime('today') - pd.datetime(1970,1,1)).days - daysShown
    cursor = db.macros.find( { 'date': { '$gt': earliestDate } } )

    #add each entry's fields to the respective lists
    for document in cursor:
        daysList.append(document['date'])
        caloriesList.append(document['macros']['calories'])
        fatList.append(document['macros']['fat'])
        carbsList.append(document['macros']['carbohydrates'])
        proteinList.append(document['macros']['protein'])
        weightList.append(document['weight'])
        sleepList.append((document['sleep']['hours'], document['sleep']['minutes']))
        print(document)

    combinedSleepList = []
    for (hours, minutes) in sleepList:
        combinedSleepList.append(hours+minutes/60.)
    

{u'weight': 159.6, u'macros': {u'carbohydrates': 271.7, u'calories': 2873.5, u'protein': 217.625, u'fat': 98.85}, u'sleep': {u'hours': 8, u'minutes': 16}, u'bodyfat%': 12, u'date': 16854, u'_id': ObjectId('56cd6f4ea93ba342a9ef2143')}
{u'weight': 161, u'macros': {u'carbohydrates': 408.1658388608, u'calories': 2939.6031071043826, u'protein': 187.52226710891875, u'fat': 62.662329194307425}, u'sleep': {u'hours': 8, u'minutes': 46}, u'bodyfat%': 12, u'date': 16855, u'_id': ObjectId('56ced2daa93ba34afe8ef1e5')}
{u'weight': 162, u'macros': {u'carbohydrates': 406.2862448256, u'calories': 3022.0808527527356, u'protein': 168.33289958609743, u'fat': 82.04893122413108}, u'sleep': {u'hours': 8, u'minutes': 45}, u'bodyfat%': 12, u'date': 16856, u'_id': ObjectId('56d01e79a93ba326e6db3362')}
{u'weight': 160.4, u'macros': {u'carbohydrates': 269.5, u'calories': 2662.5, u'protein': 159.05, u'fat': 108.75}, u'sleep': {u'hours': 8, u'minutes': 33}, u'bodyfat%': 12, u'date': 16857, u'_id': ObjectId('56d2470

## Graphing.
This cell can be used to graph your data!

In [28]:
try:
  caloriesList, dt
except NameError:
  print "Make sure to run the initial setup and data retrieval cells before this one."
else:
    today = dt.date.today()

    f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)
    ax1.plot(daysList, caloriesList, color='y')
    ax2.plot(daysList, weightList, color='b')
    ax3.plot(daysList, carbsList, color='r')
    ax4.plot(daysList, combinedSleepList, color='g')

    ax1.set_ylim([0, max(caloriesList)*1.4])
    ax2.set_ylim([min(weightList), max(weightList)*1.1])
    ax3.set_ylim([0, max(carbsList)*1.4])
    ax4.set_ylim([min(combinedSleepList), max(combinedSleepList)*1.3])

    ax1.set_title('Calories')
    ax2.set_title('Weight')
    ax3.set_title('Carbohydrates')
    ax4.set_title('Sleep')


    plt.savefig(`today.month` + "-" + `today.day` + "-" + `today.year` + ".png")


## Useful Calculations
The following cell may be useful for doing initial and recurring macronutrient calculations.

In [25]:
try:
  Male, Female
except NameError:
  print "Make sure to run the initial setup cell before this one."
else:
    macronutrientName = "Carbohydrates"
    foodName = "Sweet_Potato_1oz"
    weight = 162
    heightInIn = 72
    gender = Male
    age = 22
    activityLevel = Moderate
    goal = Bulking
    
try:
    total
except NameError:
    print "Make sure to run the tracking cell before running this one."
else:
    calories = calculateDailyCalories(weight, heightInIn, age, gender, activityLevel, goal)
    gramsOfMacroLeftToConsume = calculateGramsOfMacroLeft(calories, total, macronutrientName, weight, activityLevel)
    gramsOfMacrosTotal = calculateGramsOfMacro(calories, total, macronutrientName, weight, activityLevel)

    print "Daily calories: " + `calories`
    print "Grams of " + `macronutrientName` + " left: " + `gramsOfMacroLeftToConsume`
    print "Additional servings of " + `foodName` + " to meet daily " + `macronutrientName` + " intake: " + `calculateServingsOfFoodLeftByMacro(calories, gramsOfMacrosTotal, total, macronutrientName, foodName)`

Daily calories: 3022.6564632
Grams of 'Carbohydrates' left: 164.0093047444
Additional servings of 'Sweet_Potato_1oz' to meet daily 'Carbohydrates' intake: 28.926321077854986


In [17]:
print gramsOfMacrosTotal

456.2706402
